# this script can train a network on the pcam dataset.
Imports


In [35]:
import json
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import HDF5Matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
import argparse


In [36]:
import os
if os.getcwd().endswith('code'):
    os.chdir('..')
print(os.getcwd())


/home/j/test_pcam/repo


build arparser


In [37]:
parser = argparse.ArgumentParser()

# define the training
parser.add_argument('-lr', '--learning-rate', type=float, help='', default = 0.1)
parser.add_argument('-b','--batch-size', type=int, help='', default = 64)
parser.add_argument('--num-of-epochs', type=int, help='', default = 2)

# define the model structure
parser.add_argument('--activation-function', type=str, help='', default = 'relu')
parser.add_argument('--use-same-padding', action='store_true')
parser.add_argument('--kernel-width', type=int, help='', default = 3)
parser.add_argument('--average-kernels', type=int, help='', default = 32)
parser.add_argument('--num-of-conv-layers', type=int, help='', default = 5)
parser.add_argument('--kernel-increasing-factor', type=float, help='', default = 1.2)
parser.add_argument('--maxpool-after-n-layer', type=int, help='', default = 3)
parser.add_argument('--dropout-factor-after-conv', type=float, help='', default = 0.1)
parser.add_argument('--dropout-factor-after-maxp', type=float, help='', default = 0.25)

# define the input
parser.add_argument('--flip-input', action='store_true')
parser.add_argument('--normalize-input', action='store_true')
parser.add_argument('--use-cropping', action='store_true')

validation_steps=20
steps_per_epoch=100


In [4]:
args = parser.parse_args()


usage: ipykernel_launcher.py [-h] [-lr LEARNING_RATE] [-b BATCH_SIZE]
                             [--num-of-epochs NUM_OF_EPOCHS]
                             [--activation-function ACTIVATION_FUNCTION]
                             [--use-same-padding]
                             [--kernel-width KERNEL_WIDTH]
                             [--average-kernels AVERAGE_KERNELS]
                             [--num-of-conv-layers NUM_OF_CONV_LAYERS]
                             [--kernel-increasing-factor KERNEL_INCREASING_FACTOR]
                             [--maxpool-after-n-layer MAXPOOL_AFTER_N_LAYER]
                             [--dropout-factor-after-conv DROPOUT_FACTOR_AFTER_CONV]
                             [--dropout-factor-after-maxp DROPOUT_FACTOR_AFTER_MAXP]
                             [--flip-input] [--normalize-input]
                             [--use-cropping]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1000/jupyter/kernel-ae47c4cf-e5f5-4d9c-937a-

SystemExit: 2

In [38]:
class Args:
    def __init__(self):
        print()
        
args = Args()

args.learning_rate = 0.1
args.batch_size = 64
args.num_of_epochs = 3

args.activation_function = 'relu' ##########
args.use_same_padding = True ##############
args.kernel_width = 3 ######################
args.average_kernels = 64 ###########
args.kernel_increasing_factor = 1.5 ###########
args.num_of_conv_layers = 5 #############################
args.dropout_factor_after_conv = 0.1 ##########
args.dropout_factor_after_maxp = 0.25 #########

args.maxpool_after_n_layer = 2

args.flip_input = True
args.normalize_input = True
args.use_cropping = True

validation_steps=2
steps_per_epoch=10

#  dvc-cc dummy new -d data/camelyonpatch_level_2_split_train_x.h5 -d data/camelyonpatch_level_2_split_train_y.h5 -d data/camelyonpatch_level_2_split_valid_x.h5 -d data/camelyonpatch_level_2_split_valid_y.h5 -o tf_models/lr<<<learning_rate>>>_bz<<<batch_size>>>_<<<activation_function>>>_same<<<use_same_padding>>>_kw<<<kernel_width>>>_ak<<<average_kernels>>>_if<<<kernel_increasing_factor>>>_n<<<num_of_conv_layers>>>_dfc<<<dropout_factor_after_conv>>>_dfm<<<dropout_factor_after_maxp>>>_m<<<maxpool_after_n_layer>>>_fl<<<flip_input>>>_no<<<normalize_input>>>_cr<<<use_cropping>>>.h5 -o tensorboards/lr<<<learning_rate>>>_bz<<<batch_size>>>_<<<activation_function>>>_same<<<use_same_padding>>>_kw<<<kernel_width>>>_ak<<<average_kernels>>>_if<<<kernel_increasing_factor>>>_n<<<num_of_conv_layers>>>_dfc<<<dropout_factor_after_conv>>>_dfm<<<dropout_factor_after_maxp>>>_m<<<maxpool_after_n_layer>>>_fl<<<flip_input>>>_no<<<normalize_input>>>_cr<<<use_cropping>>> -f dvc/train.dvc --no-exec python code/train.py --lr <<<learning_rate>>> --b <<<batch_size>>> --activation-function <<<activation_function>>> --use-same-padding <<<use_same_padding>>> --kernel-width <<<kernel_width>>> --average-kernels <<<average_kernels>>> --kernel-increasing-factor <<<kernel_increasing_factor>>> --num-of-conv-layers <<<num_of_conv_layers>>> --dropout-factor-after-conv <<<dropout_factor_after_conv>>> --dropout-factor-after-maxp <<<dropout_factor_after_maxp>>> --maxpool-after-n-layer <<<maxpool_after_n_layer>>> --flip-input <<<flip_input>>> --normalize-input <<<normalize_input>>> --use-cropping <<<use_cropping>>>

#  lr<<<learning_rate>>>_bz<<<batch_size>>>_<<<activation_function>>>_same<<<use_same_padding>>>_kw<<<kernel_width>>>_ak<<<average_kernels>>>_if<<<kernel_increasing_factor>>>_n<<<num_of_conv_layers>>>_dfc<<<dropout_factor_after_conv>>>_dfm<<<dropout_factor_after_maxp>>>_m<<<maxpool_after_n_layer>>>_fl<<<flip_input>>>_no<<<normalize_input>>>_cr<<<use_cropping>>>
#  --lr <<<learning_rate>>> --b <<<batch_size>>> --activation-function <<<activation_function>>> --use-same-padding <<<use_same_padding>>> --kernel-width <<<kernel_width>>> --average-kernels <<<average_kernels>>> --kernel-increasing-factor <<<kernel_increasing_factor>>> --num-of-conv-layers <<<num_of_conv_layers>>> --dropout-factor-after-conv <<<dropout_factor_after_conv>>> --dropout-factor-after-maxp <<<dropout_factor_after_maxp>>> --maxpool-after-n-layer <<<maxpool_after_n_layer>>> --flip-input <<<flip_input>>> --normalize-input <<<normalize_input>>> --use-cropping <<<use_cropping>>>



Load the dataset:

In [39]:
x_train = HDF5Matrix('data/camelyonpatch_level_2_split_train_x.h5', 'x')
y_train = HDF5Matrix('data/camelyonpatch_level_2_split_train_y.h5', 'y')
x_valid = HDF5Matrix('data/camelyonpatch_level_2_split_valid_x.h5', 'x')
y_valid = HDF5Matrix('data/camelyonpatch_level_2_split_valid_y.h5', 'y')


define the model


In [40]:
if args.use_same_padding:
    padding = 'same'
else:
    padding = 'valid'

kernel2d = (args.kernel_width, args.kernel_width)


In [41]:
model = Sequential()
for i in range(args.num_of_conv_layers):
    kernels = args.average_kernels * (args.kernel_increasing_factor ** (i-(args.num_of_conv_layers/2.)))
    kernels = int(kernels+0.5)
    
    if i == 0:
        input_shape = list(x_train.shape[1:])
        if args.use_cropping:
            input_shape[0] -= 10
            input_shape[1] -= 10 
        
        model.add(Conv2D(kernels, kernel2d, padding=padding,
                 input_shape=input_shape))
    else:
        model.add(Conv2D(kernels, kernel2d, padding=padding))
    model.add(Activation(args.activation_function))
    if args.maxpool_after_n_layer > 0 and (i+1) % args.maxpool_after_n_layer == 0:
        model.add(MaxPooling2D(pool_size=(2, 2)))
        if args.dropout_factor_after_maxp > 0:
            model.add(Dropout(args.dropout_factor_after_maxp))
    elif args.dropout_factor_after_conv > 0:
        model.add(Dropout(args.dropout_factor_after_conv))

model.add(GlobalAveragePooling2D())
if args.dropout_factor_after_maxp > 0:
    model.add(Dropout(args.dropout_factor_after_maxp))


model.add(Flatten())
model.add(Dense(2))
model.add(Activation('softmax'))


In [42]:
optimizer = tf.keras.optimizers.Adam(args.learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy',tf.keras.metrics.AUC()])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 86, 86, 23)        644       
_________________________________________________________________
activation_6 (Activation)    (None, 86, 86, 23)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 86, 86, 23)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 86, 86, 35)        7280      
_________________________________________________________________
activation_7 (Activation)    (None, 86, 86, 35)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 43, 43, 35)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 43, 43, 35)       

data loader


In [43]:
def next_data_pcam(x,y,bz=args.batch_size):
    datalen = len(x)
    while True:
        indizies = None
        while indizies is None or len(indizies) == bz:
            indizies = np.unique(sorted(np.random.randint(datalen,size=bz)))
        
        x_data = np.array(x[indizies])
        if args.normalize_input:
            x_data = x_data/256.0
        if args.use_cropping:
            r = np.random.randint(10)
            r2 = np.random.randint(10)
            x_data = x_data[:,r:-10+r,r2:-10+r2]
        if args.flip_input:
            if np.random.randint(2) == 1:
                x_data = x_data[:,::-1]
            if np.random.randint(2) == 1:
                x_data = x_data[:,:,::-1]
        
        yield x_data, np.array([[1,0],[0,1]])[y[indizies][:,0,0,0]]
      

In [12]:
tmp = next(next_data_pcam(x_train, y_train))
print(tmp[0].shape)
print(tmp[1].shape)
        

(63, 86, 86, 3)
(63, 2)


train the model


In [44]:
if not os.path.exists('tensorboards'):
    os.mkdir('tensorboards')


In [45]:
tensorboard = tf.keras.callbacks.TensorBoard('tensorboards/tb')
history = model.fit_generator(next_data_pcam(x_train, y_train),
                        validation_steps=10,
                        steps_per_epoch=100,
                        epochs=args.num_of_epochs,
                        validation_data=next_data_pcam(x_valid, y_valid),
                        workers=1, verbose=2,
                        callbacks=[tensorboard])


Epoch 1/3
10/10 - 16s - loss: 6.8758 - accuracy: 0.5222 - auc_1: 0.5299 - val_loss: 7.8032 - val_accuracy: 0.5159 - val_auc_1: 0.5159
Epoch 2/3
10/10 - 10s - loss: 7.8800 - accuracy: 0.5111 - auc_1: 0.5111 - val_loss: 6.9078 - val_accuracy: 0.5714 - val_auc_1: 0.5714
Epoch 3/3
10/10 - 9s - loss: 7.4450 - accuracy: 0.5381 - auc_1: 0.5381 - val_loss: 8.1870 - val_accuracy: 0.4921 - val_auc_1: 0.4921


In [46]:
if not os.path.exists('tf_models'):
    os.mkdir('tf_models')
model.save_weights('./tf_models/tf_model.h5')



Just some pseudo output



In [17]:
import matplotlib.pyplot as plt
import numpy as np
size = 100000
samples = np.random.beta(1,5,size=size)
plt.hist(samples, bins=100, density=True)
x1,x2,y1,y2 = plt.axis()
plt.axis((0.0,1.0,y1,y2))
# axes.set_ylim([ymin,ymax])


(0.0, 1.0, 0.0, 5.1617353767212855)

In [ ]:
# !pip install seaborn



In [56]:
# save all history data

if not os.path.exists('outputs'):
    os.mkdir('outputs')

with open('outputs/all-history.json','w') as f:
    json.dump(str(history.history),f)



In [57]:
params = {}
for p in history.history:
    if p.find('loss') >= 0:
        params[p] = np.min(history.history[p])
    else:
        params[p] = np.max(history.history[p])
        
with open('outputs/history-summary.json','w') as f:
    json.dump(str(params),f)







{'loss': 6.875818032026291, 'accuracy': 0.53809524, 'auc_1': 0.53809524, 'val_loss': 6.907755374908447, 'val_accuracy': 0.5714286, 'val_auc_1': 0.57142854}
